In [1]:

#dependencies
import requests
import json
import pandas as pd
import gmaps

# Google API Key
from config import gkey

# import gmaps.datasets

# Configure gmaps
# gmaps.configure(api_key=gkey)



In [2]:
zip_codes = ["75001","75006","75019","75038","75039","75040","75041","75042","75043","75044","75048","75050","75051","75052","75054","75060","75061","75062","75063","75080","75081","75082","75088","75089","75104","75115","75116","75134","75137","75141","75146","75149","75150","75159","75172","75180","75181","75182","75201","75202","75203","75204","75205","75206","75207","75208","75209","75210","75211","75212","75214","75215","75216","75217","75218","75219","75220","75223","75224","75225","75226","75227","75228","75229","75230","75231","75232","75233","75234","75235","75236","75237","75238","75240","75241","75243","75244","75246","75247","75248","75249","75251","75253","75254","75270","75390"]
zip_code_details = []

census = pd.read_csv("Resources/DallasCensusData.csv")


gmaps.configure(api_key=gkey)
# Target city
def get_coordinates_per_zipcode(zip_code):
    target_county = zip_code
    # Build the endpoint URL
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
        'address={0}&key={1}').format(target_county, gkey)
    data = requests.get(target_url).json()
    return data
for zip_code in zip_codes:
    data = get_coordinates_per_zipcode(zip_code)
    code = int(data["results"][0]["address_components"][0]["long_name"])
    lat = data["results"][0]["geometry"]["location"]["lat"]
    lng = data["results"][0]["geometry"]["location"]["lng"]
    zip_code_details.append({"zip_code": code, "lat": lat, "lng":lng})


In [3]:
zips_df = pd.DataFrame(zip_code_details, columns = ["zip_code","lat","lng"])
zips_df.head()

,zip_code,lat,lng
0,75001,32.965478,-96.829740
1,75006,32.967845,-96.888720
2,75019,32.961876,-96.996092
3,75038,32.872152,-96.996092
4,75039,32.891677,-96.947753


In [4]:
zips_df.to_csv("ZipcodeGeoLocation.csv", index=False)

In [5]:
zip_merge = pd.merge(census,zips_df,left_on='Zipcode', right_on='zip_code', how='inner')
dropped_data = zip_merge.dropna(how="any")

dropped_data.head()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Count,City,zip_code,lat,lng
0,75001,13214.0,32.2,57633.0,43919.0,1722.0,13.031633,433.0,Addison,75001,32.965478,-96.829740
1,75006,47590.0,34.8,56985.0,27010.0,6927.0,14.555579,1996.0,Carrollton,75006,32.967845,-96.888720
2,75019,39551.0,37.9,110445.0,48040.0,1624.0,4.106091,985.0,Coppell,75019,32.961876,-96.996092
3,75038,28161.0,30.0,48130.0,28391.0,4581.0,16.267178,1512.0,Irving,75038,32.872152,-96.996092
4,75039,12197.0,32.2,73503.0,65135.0,819.0,6.714766,264.0,Irving,75039,32.891677,-96.947753


In [6]:


# Store latitude and longitude in locations
locations = dropped_data[["lat", "lng"]]

#create weights by poverty rate
weights = dropped_data['Poverty Rate']






In [7]:
# Plot Heatmap
fig = gmaps.figure()

# Add layer
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, max_intensity=20))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))